In [1]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F

# Data Pre-Processing 

Read in the training file


In [2]:
def openFile(file): 
    '''
    Description: Read in text file from CLI and return the input string and label in seperate arrays
    Input: file 
    Output: Two arrays, input and target, containing the input and target strings respectively 
    '''
    input = []
    output = []
    with open(file, 'r') as f:
        for line in f: 
            if ',' in line:
                currentLine = line.split(',')
                input.append(currentLine[0])
                output.append(currentLine[1].strip('\n'))
            else:
                input.append(line.strip('\n'))
    return input, output

In [3]:
def writeFile(input, output):
    '''
    Description: write the input and predicted translation to a text file
    Input: Original input to model
    Output: Output of model
    Return: None, text file stored in file path where code is exectued in CLI
    '''
    with open('result_predict.txt', 'a') as f:
        for i in input: 
            f.write(input[i])
            f.write(',')
            f.write(output[i])
            f.write('\n')
    return

In [4]:
input, output = openFile('data_train.txt')

In [5]:
max_length = len(max(output, key=len))

In [6]:
predictInput, _ = openFile('data_predict.txt')

Combine into pairs

In [7]:
sos_tkn = 0
eos_tkn = 1

class Sequence:
    '''
    Class to help make a dictionary. Each word from the input or target array is split into letters
    and then added to the dictionary. Each letter is added to an index
    '''
    def __init__(self): 
    #initialize containers to hold the letters and corresponding index
        self.letter2index = {}
        self.letter2count = {}
        self.index2letter = {}
        self.n_letters = 2
    
    def addWord(self, word): 
         '''
         Description: split a word into letters and pass to addLetter function
         Input: word from input or target sequence
         Output: none 
         '''
         for letter in word:
            self.addLetter(letter)
     
    def addLetter(self, letter): 
         if letter not in self.letter2index:
             self.letter2index[letter] = self.n_letters
             self.letter2count[letter] = 1
             self.index2letter[self.n_letters] = letter
             self.n_letters += 1
         else:
             self.letter2count[letter] += 1

In [8]:
def pairData(input, output): 

    source = Sequence()
    target = Sequence()

    pairs = []

    for i in range(len(input)):
        full = [input[i], output[i]]
        source.addWord(input[i])
        target.addWord(output[i])

        pairs.append(full)

    return source, target, pairs

In [9]:
def loadPredict(input):
    predict = Sequence()

    for i in range(len(input)):
        predict.addWord(input[i])
    
    return predict

In [10]:
source, target, pairs = pairData(input, output)
print(random.choice(pairs))

['GILGFVFTL', 'CAENGGGSTLGRLYF']


In [11]:
predict = loadPredict(predictInput)

Convert to input tensors

In [12]:
def indexesFromWord(Sequence, word):
    return [Sequence.letter2index[letter] for letter in word]

def tensorFromWord(Sequence, word):
    indexes = indexesFromWord(Sequence, word)
    indexes.append(eos_tkn)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1,1)

def tensorsFromPair(pair):
    input_tensor = tensorFromWord(source, pair[0])
    target_tensor = tensorFromWord(target, pair[1])
    return (input_tensor, target_tensor)

def tensorsForPredict(input): 
    predict_tensor = tensorFromWord(predict, predictInput)
    return predict_tensor

#The Model


In [13]:
class Encoder(nn.Module): 
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded 
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self): 
        return torch.zeros(1,1, self.hidden_size, device=device)



In [14]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=max_length):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


#Training

In [15]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=max_length):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[sos_tkn]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == eos_tkn:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Timer

In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plot Results

In [18]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

#Evaluation

In [19]:
def evaluate(encoder, decoder, sentence, max_length=max_length):
    with torch.no_grad():
        input_tensor = tensorFromWord(source, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[sos_tkn]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == eos_tkn:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(target.index2letter[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_letters, attentions = evaluate(encoder, decoder, pair[0])
        output_word = ''.join(output_letters)
        print('<', output_word)
        print('')

In [21]:
def evalOnPred(encoder, decoder, word, max_length=max_length):
     with torch.no_grad(): 
        input_tensor = tensorFromWord(predict, word)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                        encoder_hidden)
                encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[sos_tkn]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == eos_tkn:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(target.index2letter[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [22]:
def evaluatePredictionData(encoder, decoder):
    with open('result_predict.txt', 'a') as f:
        for i in range(len(predictInput)):
            print('>', predictInput[i])
            output_letters, attentions = evalOnPred(encoder, decoder, predictInput[i])

            
            output_word = ''.join(output_letters)
            print('=', output_word.strip('<EOS>'))

            
            
            f.write(predictInput[i])
            f.write(',')
            f.write(output_word.strip('<EOS>'))
            f.write('\n')
            

In [23]:
hidden_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder1 = Encoder(source.n_letters, hidden_size).to(device)
attn_decoder1 = AttnDecoder(hidden_size, target.n_letters, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 100000, print_every=5000)

#save models
torch.save(encoder1.state_dict(), 'encoder_model_100th.pt')
torch.save(attn_decoder1.state_dict(), 'decoder_model_100th.pt' )

4m 28s (- 85m 6s) (5000 5%) 1.7891
9m 9s (- 82m 25s) (10000 10%) 1.6741
13m 30s (- 76m 31s) (15000 15%) 1.6391
18m 4s (- 72m 19s) (20000 20%) 1.6354
22m 40s (- 68m 2s) (25000 25%) 1.6234
27m 10s (- 63m 24s) (30000 30%) 1.6269
31m 47s (- 59m 1s) (35000 35%) 1.6100
36m 9s (- 54m 13s) (40000 40%) 1.5931
40m 43s (- 49m 46s) (45000 45%) 1.6063
45m 7s (- 45m 7s) (50000 50%) 1.5846
49m 30s (- 40m 30s) (55000 55%) 1.5968
53m 49s (- 35m 53s) (60000 60%) 1.5807
58m 30s (- 31m 30s) (65000 65%) 1.5739
63m 10s (- 27m 4s) (70000 70%) 1.5845
67m 44s (- 22m 34s) (75000 75%) 1.5685
72m 9s (- 18m 2s) (80000 80%) 1.5704
76m 33s (- 13m 30s) (85000 85%) 1.5783
80m 57s (- 8m 59s) (90000 90%) 1.5526
85m 20s (- 4m 29s) (95000 95%) 1.5641
90m 4s (- 0m 0s) (100000 100%) 1.5589


#Load Models (optional)


In [ ]:
encoder1.load_state_dict(torch.load('encoder_model.pt', map_location=device))
encoder1.eval()

attn_decoder1.load_state_dict(torch.load('decoder_model.pt', map_location=device))
attn_decoder1.eval()

AttnDecoder(
  (embedding): Embedding(22, 256)
  (attn): Linear(in_features=512, out_features=24, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=22, bias=True)
)

#Evaluate and Write To Output

In [24]:
evaluateRandomly(encoder1, attn_decoder1)

> NLVPMVATV
= CASMNDGATYEQYF
< CASSLSTGGGYFFFF<EOS>

> NLNCCSVPV
= CASSIVGQGYEQYF
< CASSLGGGYEYYFFF<EOS>

> LLAMKVPNV
= CASSQDNNEQFF
< CASSLGQGGYEQYF<EOS>

> GILGFVFTL
= CILRDPFTGGGIKFTF
< CASSLRSGGEQFFF<EOS>

> GLCTLVAML
= CASSLTDPYEQFF
< CASSLSGGGGEQFF<EOS>

> NLVPMVATV
= CASSAGGGSYNEQFF
< CASSLSTGGTYFFFF<EOS>

> GILGFVFTL
= CASSVYSNQPQHFT
< CASSLRSGGEQFFF<EOS>

> GILGFVFTL
= CASSLAGSSDEQFF
< CASSLRSGGEQFFF<EOS>

> FLYALALLL
= CASSVGGHLNEKLFF
< CASSQQGGGGYTFFFF<EOS>

> EAAGIGILTV
= CASSETGGTEAF
< CASSLSGGGGEQFFF<EOS>



In [26]:
evaluatePredictionData(encoder1, attn_decoder1)

> SMLGIGIYPV
= CASSLSGGGEQYF
> LMFDRGMSLL
= CASSSGGGGGYFFFF
> ILEDRGFNQV
= CASSLSLGGEQYF
> MMWDRGLGMM
= CASSSGSGGGYFFFFF
> NLSNLGILPV
= CASSLSGGGEQYFF
> IMEDVGWLNV
= CASSLSLGGEQYF
> NMGGLGIMPV
= CASSLSGGGEQFF
> MMWDRGMGLL
= CASSSSSGGEQYFF
> SMAGIGIVDV
= CASSLSGGGEQYF
> SMLGIGIVPV
= CASSLSGGGEQYF
> ALDSRSEHFM
= CASSSGGGYNEQFF
> AMDSRADMFV
= CASSLGGGGYYFFFF
> SMNSREEVFV
= CASSLSGGEQYFF
> IMDSKSENFL
= CASSFSGGEQEFF
> SMNSHSGTFL
= CASSSGGGGEQFF
> AAGIGILTV
= CASSLSGGEQYYFFF
> AAGIGILTV
= CASSLSGGEQYYFFF
> ALFNTVATL
= CASSLSGGGEQFFF
> CLFNTVATL
= CASSLSGGGEQFFF
> DLFNTVATL
= CASSLSGGGEQFFF
> ELFNTVATL
= CASSLSGGGEQFFF
> FLFNTVATL
= CASSLSGGGEQFF
> GLFNTVATL
= CASSLSGGGEQFF
> HLFNTVATL
= CASSLSGGGEQFFF
> ILFNTVATL
= CASSLSGGGEQFFF
> KLFNTVATL
= CASSLSGGGEQFFF
> LLFNTVATL
= CASSLSGGGEQFFF
> MLFNTVATL
= CASSLSGGGEQFFF
> NLFNTVATL
= CASSLSGGGEQFFF
> PLFNTVATL
= CASSLSGGGEQFFF
> QLFNTVATL
= CASSLSGGGEQFFF
> RLFNTVATL
= CASSLSGGGEQFFF
> TLFNTVATL
= CASSLSGGGEQFF
> VLFNTVATL
= CASSLSGGGEQFFF
> WLF